# Explore Pose Estimation with Mediapipe

## https://pypi.org/project/mediapipe/

## https://google.github.io/mediapipe/

### This is an optimised multiplatform __FREE__ opensource package with Python bindings, meaning you can use it in Python.

Do note that some functions are not available in Python yet.

You can `pip` install it:

```
python -m pip install mediapipe
```

In [ ]:
import cv2  # OpenCV. Is always called 'cv2' for some reason. Presumably historically.
import mediapipe as mp  # MediaPipe - the pose estimation engine.
import time  # An in-built Python library managing Time functions. 
import numpy as np

mpPose = mp.solutions.pose
pose = mpPose.Pose()
mpDraw = mp.solutions.drawing_utils

In [ ]:
cap = cv2.VideoCapture(0)
#cap = cv2.VideoCapture('a.mp4')

In [ ]:
# Body parts list aas per: https://google.github.io/mediapipe/solutions/pose.html

BODY_PARTS ={"Nose":0,
			 "L_eye_inner":1,
			 "L_eye":2,
			 "L_eye_outer":3,
			 "R_eye_inner":4,
			 "R_eye":5,
			 "R_eye_outer":6,
			 "L_ear":7,
			 "R_ear":8,
			 "L_mouth":9,
			 "R_mouth":10,
			 "L_shoulder":11,
			 "R_shoulder":12,
			 "L_elbow":13,
			 "R_elbow":14,
			 "L_wrist":15,
			 "R_wrist":16,
			 "L_pinky":17,
			 "R_pinky":18,
			 "L_index":19,
			 "R_index":20,
			 "L_thumb":21,
			 "R_thumb":22,
			 "L_hip":23,
			 "R_hip":24,
			 "L_knee":25,
			 "R_knee":26,
			 "L_ankle":27,
			 "R_ankle":28,
			 "L_heel":29,
			 "R_heel":30,
			 "L_foot_index":31,
			 "R_foot_index":32,
			 }

BODY_PARTS_ID = {}
for key in BODY_PARTS:
	BODY_PARTS_ID[BODY_PARTS[key]] = key


In [ ]:

coords_x = np.zeros(3)
coords_y = np.zeros(3)

pTime = 0


In [ ]:

while True:
	success, img = cap.read()
	imgRGB = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
	results = pose.process(imgRGB)
	# print(results.pose_landmarks)
	if results.pose_landmarks:
		mpDraw.draw_landmarks(img, results.pose_landmarks, mpPose.POSE_CONNECTIONS)
		for id, lm in enumerate(results.pose_landmarks.landmark):
			h, w,c = img.shape
			# print(id, lm)
			cx, cy = int(lm.x*w), int(lm.y*h)
			cv2.circle(img, (cx, cy), 5, (255,0,0), cv2.FILLED)
			cv2.putText(img, str(BODY_PARTS_ID[id]), (cx+5,cy+10), cv2.FONT_HERSHEY_SIMPLEX,0.8,(220,220,220), 3)

		r_shoulder = results.pose_landmarks.landmark[BODY_PARTS['R_shoulder']]
		l_shoulder = results.pose_landmarks.landmark[BODY_PARTS['L_shoulder']]
		sh_x = int(w * (r_shoulder.x + l_shoulder.x) / 2)
		sh_y = int(h * (r_shoulder.y + l_shoulder.y) / 2)

		r_hip = results.pose_landmarks.landmark[BODY_PARTS['R_hip']]
		l_hip = results.pose_landmarks.landmark[BODY_PARTS['L_hip']]
		hip_x = int(w * (r_hip.x + l_hip.x) / 2)
		hip_y = int(h * (r_hip.y + l_hip.y) / 2)

		r_ankle = results.pose_landmarks.landmark[BODY_PARTS['R_ankle']]
		l_ankle = results.pose_landmarks.landmark[BODY_PARTS['L_ankle']]
		ank_x = int(w * (r_ankle.x + l_ankle.x) / 2)
		ank_y = int(h * (r_ankle.y + l_ankle.y) / 2)

		cv2.line(img, (sh_x, sh_y), (hip_x, hip_y), (0,256,0), 3)
		cv2.line(img, (hip_x, hip_y), (ank_x, ank_y), (0,256,0), 3)


		coords_x = np.array([sh_x, hip_x, ank_x])
		coords_y = np.array([sh_y, hip_y, ank_y])
		box_width = coords_x.max() - coords_x.min()
		box_height = coords_y.max() - coords_y.min()
		box_ratio = box_height / box_width
		if box_ratio < 0.5:
			print('FALL!')
			cv2.rectangle(img,
			              (coords_x.min(),coords_y.max() ),
			              (coords_x.max(),coords_y.min() ),
						  (0,0,255),
						  5
						  )

	# Keep timing code together for simplicity
	cTime = time.time()  # Update Current frame time
	fps = 1/(cTime-pTime)  # Calculate framerate
	pTime = cTime  # Update 'previous' frame time

	cv2.putText(img, str(int(fps)), (50,50), cv2.FONT_HERSHEY_SIMPLEX,1,(255,0,0), 3)
	cv2.imshow("Image", img)
	cv2.waitKey(1)
